<a href="https://colab.research.google.com/github/SarahPendhari/TumorSense/blob/main/Brain_Tumor_Detection_Comparative_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
import os

path = os.listdir(r'/content/drive/MyDrive/Dataset/Training')


In [3]:
classes = {'no_tumor' : 0 , 'pituitary_tumor' : 1}
classes

{'no_tumor': 0, 'pituitary_tumor': 1}

In [ ]:
import cv2
X = []
Y = []
for cls in classes:
    pth = r'/content/drive/MyDrive/Dataset/Training//'+cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j, 0)
        img = cv2.resize(img, (200,200))
        X.append(img)
        Y.append(classes[cls])

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
np.unique(Y) # 0 - No Tumor(Negative), 1 - Tumor(Positive)

In [ ]:
pd.Series(Y).value_counts()         #Calculating the count of both the categories

In [ ]:
plt.imshow(X[10], cmap='gray')

In [ ]:
X_updated = X.reshape(len(X), -1)
X_updated.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X_updated, Y, random_state=42,test_size=.20)

In [ ]:
xtrain.shape, xtest.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report


clf = DecisionTreeClassifier()
clf.fit(xtrain, ytrain)

# Make predictions on the test
ypred = clf.predict(xtest)

# Evaluate the model
accuracy = accuracy_score(ytest, ypred)
report = classification_report(ytest, ypred)
print(report)
# Print the results
print(f"Accuracy: {accuracy}")
# Assuming your model is trained with 40000 features per sample
# Adjust the resize dimensions accordingly
image = cv2.imread('/content/drive/MyDrive/Dataset/Training/meningioma_tumor/m (6).jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (200, 200))  # Adjusted to create 40000 features (200x200)

pixels = image.reshape(-1)
if len(pixels) == 40000:
    pixels2d = [pixels]
    predicted_class = clf.predict(pixels2d)
    print(f"Predicted Class: {predicted_class[0]}")
else:
    print("Error: Mismatch in feature dimensions.")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(15, 10))
plot_tree(clf, filled=True, feature_names=None, class_names=["No Tumor","Tumor"])
plt.savefig('decision_tree.png')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# Assuming ypred and ytest are already defined as per your previous code

# Calculate confusion matrix
conf_matrix = confusion_matrix(ytest, ypred)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)
print('\n')
normalized_conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Print the normalized confusion matrix
print("Normalized Confusion Matrix - Decision Tree:")
print(normalized_conf_matrix)

# Plot the normalized confusion matrix with a blue color map
plt.figure(figsize=(10, 7))
sns.heatmap(normalized_conf_matrix, annot=True, fmt=".2f", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalized Confusion Matrix- Decision Tree:')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

# Assuming you have a binary classification and ytest, xtest are your test labels and features
# Compute probabilities
y_prob = clf.predict_proba(xtest)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(ytest, y_prob)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
errors = []
thresholds = np.linspace(0, 1, 100)

for thresh in thresholds:
    errors.append(np.mean(((y_prob > thresh) * 1) != ytest))

plt.figure()
plt.plot(thresholds, errors, label='Error rate')
plt.xlabel('Threshold')
plt.ylabel('Error rate')
plt.title('Error Rate vs. Threshold')
plt.legend()
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Create a Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42) # n_estimators is the number of trees

# Train the model on the training data
rf_clf.fit(xtrain, ytrain)


In [ ]:
# Make predictions on the test data
y_pred_rf = rf_clf.predict(xtest)

# Evaluate the model
accuracy_rf = accuracy_score(ytest, y_pred_rf)
report_rf = classification_report(ytest, y_pred_rf)

# Print the results
print(f"Random Forest Accuracy: {accuracy_rf}")
print("Random Forest Classification Report:")
print(report_rf)


In [ ]:
# Compute confusion matrix
cm = confusion_matrix(ytest, ypred)

# Normalize the confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plotting
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap='Blues',
            xticklabels=rf_clf.classes_, yticklabels=rf_clf.classes_)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Normalized Confusion Matrix for RandomForestClassifier')
plt.show()


In [ ]:
# Assuming you have your test labels in ytest and
# your model's predicted probabilities for the positive class in y_scores
y_scores = rf_clf.predict_proba(xtest)[:, 1]  # For binary classification

# Compute ROC curve and ROC area
fpr, tpr, _ = roc_curve(ytest, y_scores)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Calculate the misclassification for each threshold
misclassification = 1 - tpr

# Plotting
plt.figure()
plt.plot(fpr, misclassification, color='blue', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('Misclassification Error')
plt.title('Misclassification Error across different thresholds')
plt.show()


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(xtrain, ytrain)
# Making predictions
ypred_knn = knn_clf.predict(xtest)

# Evaluating the model
accuracy_knn = accuracy_score(ytest, ypred_knn)
report_knn = classification_report(ytest, ypred_knn)

# Print the results
print(f"Accuracy: {accuracy_knn}")
print("Classification Report:")
print(report_knn)


In [ ]:
cm_knn = confusion_matrix(ytest, ypred_knn)
cm_knn_normalized = cm_knn.astype('float') / cm_knn.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_knn_normalized, annot=True, fmt=".2f", cmap='Blues')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Normalized Confusion Matrix for KNN Classifier')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming your model and test data are named knn_classifier, Xtest, ytest
# Compute predicted probabilities: y_pred_prob
y_pred_prob = knn_clf.predict_proba(xtest)[:,1]

# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, label='KNN (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
error_rate = []
for i in range(1, 40):  # Assuming you want to test from 1 to 39 neighbors
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(xtrain, ytrain)
    pred_i = knn.predict(xtest)
    error_rate.append(np.mean(pred_i != ytest))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, 40), error_rate, color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('Number of Neighbors (K)')
plt.ylabel('Error Rate')
plt.show()


In [ ]:
from sklearn import svm
# Create a SVM classifier with a linear kernel
svm_clf = svm.SVC(kernel='linear')

# Train the model on the training data
svm_clf.fit(xtrain, ytrain)
# Predict the response for the test dataset
y_pred = svm_clf.predict(xtest)

# Calculate accuracy
accuracy = accuracy_score(ytest, y_pred)
print(f"Accuracy: {accuracy}")

# Print classification report for detailed analysis
print(classification_report(ytest, y_pred))


In [ ]:
# Compute confusion matrix
cm = confusion_matrix(ytest, y_pred)

# Normalize the confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# Create a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues")
plt.title('Normalized Confusion Matrix for SVM')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


In [ ]:

y_scores = svm_clf.decision_function(xtest)
fpr, tpr, threshold = roc_curve(ytest, y_scores)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(10, 8))
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


In [ ]:
plt.figure(figsize=(10, 8))
plt.title('Error vs. Threshold')
plt.plot(threshold, 1 - tpr, label='Error Rate')
plt.legend(loc = 'lower left')
plt.xlim([threshold.min(), threshold.max()])
plt.xlabel('Threshold')
plt.ylabel('Error Rate')
plt.show()


In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

# Load and preprocess data
def load_data(directory, classes):
    X = []
    Y = []
    for idx, cls in enumerate(classes):
        path = os.path.join(directory, cls)
        for img in os.listdir(path):
            try:
                image = cv2.imread(os.path.join(path, img))
                image = cv2.resize(image, (100, 100))  # Resize images
                X.append(image)
                Y.append(idx)
            except Exception as e:
                print(e)
    return np.array(X), np.array(Y)

# Classes
classes = ['no_tumor', 'pituitary_tumor']  # Update with your classes
data_directory = '/content/drive/MyDrive/Dataset/Training//'  # Update with your dataset path

# Load dataset
X, Y = load_data(data_directory, classes)

# Normalize the data
X = X / 255.0

# One-hot encode labels
Y = to_categorical(Y, num_classes=len(classes))

# Split the data
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(xtrain, ytrain, epochs=10, validation_data=(xtest, ytest))

# Evaluate the model
test_loss, test_acc = model.evaluate(xtest, ytest)
print(f"Test Accuracy: {test_acc}")

# Model summary
model.summary()


In [ ]:
history_dict = history.history

# Extract accuracy and loss values for training
acc = history_dict['accuracy']
loss = history_dict['loss']

# Extract accuracy and loss values for validation
val_acc = history_dict['val_accuracy']
val_loss = history_dict['val_loss']

# Range of epochs
epochs = range(1, len(acc) + 1)

# Accuracy plot
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Training Accuracy', color='blue')  # Blue for training
plt.plot(epochs, val_acc, 'go-', label='Validation Accuracy', color='green')  # Green for validation
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Training Loss', color='blue')  # Blue for training
plt.plot(epochs, val_loss, 'go-', label='Validation Loss', color='green')  # Green for validation
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
